In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from writers import getStories


In [7]:
#from rich import print

In [16]:
df = getStories()
s = "20240908"
story_details = df[df.post == s].iloc[0]
title, story = story_details.title, story_details.story


print("--> "+s+"\n\n# "+title+"\n\n"+story)

--> 20240908

# The Box of Possibilities

Margo was the last person you’d expect to be a travel influencer. With the self-control of a hyperactive squirrel on espresso and a penchant for packing enough shoes for a two-day trip to the grocery store, she had somehow managed to become a beacon of luggage-free travel. A feat not accomplished by her own virtue, mind you, but rather the result of a catastrophic ink spill on her favorite suitcase, a tragically ironic incident that led her to discover a new world of renting clothes in Japan. 

"Rent-a-Robe, Margo!" she exclaimed to her friend Lisa, who was still trying to figure out why the universe had deemed it necessary to include Margo in her life. "It’s the future! Just think about it—no more lugging around your entire wardrobe while simultaneously trying to convince the airline that your 75-pound bag is actually just a 'heavy-duty carry-on'."^1

Lisa sighed, wondering if it was too late to change her flight to a different dimension. Marg